In [1]:
from models import Baseline_MLP
import torch
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [13]:
input_shape = (1, 45, 5, 5, 5)
task = "regression"

if task == "classification":
    output_size = 7
    output_shape = (1, output_size)
elif task == "regression":
    output_size = 1
    output_shape = (1, output_size)

inputs = torch.randn(input_shape).to(device)
labels = torch.randn(output_shape).to(device)

# Get the output size of efficientnet
efficientnet_output_size = input_shape[1] * input_shape[2] * input_shape[3] * input_shape[4]

# Initialize the model
model = Baseline_MLP(efficientnet_output_size=efficientnet_output_size, hidden_size=100, output_size=output_size, task=task).to(device)

In [14]:
model = model.to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model.train()

Baseline_MLP(
  (combination_mlp): TwoInputMLP(
    (prev_pred_FC): Linear(in_features=6, out_features=100, bias=True)
    (efficientnet_FC): Linear(in_features=5625, out_features=100, bias=True)
    (combo_FC): Linear(in_features=200, out_features=1, bias=True)
  )
  (final_activation): Sigmoid()
)

In [15]:
previous_prediction_1 = torch.randn((1, 3))
previous_prediction_2 = torch.randn((1, 3))
previous_predictions = torch.cat((previous_prediction_1, previous_prediction_2), dim=1).cuda()

In [16]:
for epoch in range(2):
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs, previous_predictions)
    print("outputs", outputs)
    print("outputs.shape", outputs.shape)
    print("labels", labels)
    print("labels.shape", labels.shape)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    print('[%d] loss: %.3f' % (epoch + 1, loss.item()))

print('Finished Training')
print("outputs.shape", outputs.shape)
print("output", outputs.cpu().detach().numpy())

Previous predictions shape torch.Size([1, 100])
Efficientnet output shape torch.Size([1, 100])
outputs tensor([[206.1955]], device='cuda:0', grad_fn=<MulBackward0>)
outputs.shape torch.Size([1, 1])
labels tensor([[-0.2489]], device='cuda:0')
labels.shape torch.Size([1, 1])
[1] loss: 42619.258
Previous predictions shape torch.Size([1, 100])
Efficientnet output shape torch.Size([1, 100])
outputs tensor([[0.]], device='cuda:0', grad_fn=<MulBackward0>)
outputs.shape torch.Size([1, 1])
labels tensor([[-0.2489]], device='cuda:0')
labels.shape torch.Size([1, 1])
[2] loss: 0.062
Finished Training
outputs.shape torch.Size([1, 1])
output [[0.]]
